In [ ]:
import os
import sys

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
import chromadb
import numpy as np
import io
import requests
import json
import shutil
import time
import yaml
from io import BytesIO
from PIL import Image
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
import tarfile
load_dotenv(os.path.join(repo_dir,".env"))

## Batch image ingestion

In [ ]:
!snapi app list | grep CLIP -A 4

In [ ]:
def load_config(file_path):
    with open(file_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

config = load_config(os.path.join(kit_dir,'config.yaml'))

PENDING_RDU_JOB_STATUS = 'PENDING_RDU'
SUCCESS_JOB_STATUS = 'EXIT_WITH_0'
FAILED_JOB_STATUS = 'FAILED'

In [ ]:
class BatchClipProcessor():
    
    def __init__(self, config) -> None:
        self.headers = {
            'content-type': 'application/json',
            'key': os.getenv('SAMBASTUDIO_KEY'),
        }
        self.datasets_path = f".{config['clip']['datasets']['datasets_path']}"
        self.dataset_id = None
        self.dataset_name = config['clip']['datasets']['dataset_name']
        self.dataset_description = config['clip']['datasets']['dataset_description']
        self.dataset_source_type = config['clip']['datasets']['dataset_source_type']
        self.dataset_source_file = f".{config['clip']['datasets']['dataset_source_file']}"
        
        self.clip_app_id = config['clip']['apps']['clip_app_id']
        
        self.base_url = config['clip']['urls']['base_url']
        self.datasets_url = config['clip']['urls']['datasets_url'] 
        self.projects_url = config['clip']['urls']['projects_url'] 
        self.jobs_url = config['clip']['urls']['jobs_url'] 
        self.download_results_url = config['clip']['urls']['download_results_url'] 
    
        self.project_name = config['clip']['projects']['project_name']
        self.project_description = config['clip']['projects']['project_description']
        self.project_id=None
        
        self.job_name = config['clip']['jobs']['job_name']
        self.job_task = config['clip']['jobs']['job_task']
        self.job_type = config['clip']['jobs']['job_type']
        self.job_description = config['clip']['jobs']['job_description']
        self.model_checkpoint = config['clip']['jobs']['model_checkpoint']
        
        self.output_path = config['clip']['output']['output_path']
        
        
    def _get_call(self, url, params = None, success_message = None):
        response = requests.get(url, params=params, headers=self.headers)

        if response.status_code == 200:
            logging.info('GET request successful!')
            logging.info(success_message)
            logging.debug(f'Response: {response.text}')
        else:
            logging.error(f'GET request failed with status code: {response.status_code}')
            logging.error(f'Error message: {response.text}')
        return response

    def _post_call(self, url, params, success_message = None):
        response = requests.post(url, json=params, headers=self.headers)

        if response.status_code == 200:
            logging.info('POST request successful!')
            logging.info(success_message)
            logging.debug(f'Response: {response.text}')
        else:
            logging.error(f'POST request failed with status code: {response.status_code}')
            raise Exception(f'Error message: {response.text}')
        return response
    
    def _delete_call(self, url):
        response = requests.delete(url, headers=self.headers)    
        if response.status_code == 200:
            logging.info(f'Dataset {self.dataset_name} deleted successfully.')
            logging.debug(f'Response: {response.text}')
        else:
            logging.error(f'Failed to delete the resource. Status code: {response.status_code}')
            raise Exception(f'Error message: {response.text}')    
        return response
    def _generate_csv(self, dataset_dir):
        image_paths = []
        for root, dirs, files in os.walk(dataset_dir):
            for file in files:
                if file.endswith(('.jpg', '.jpeg', '.png', '.gif')):
                    image_path = os.path.relpath(os.path.join(root, file), dataset_dir)
                    image_paths.append(image_path)

        df = pd.DataFrame({'image_path': image_paths, 'description': '', 'subset': '', 'metadata': ''})
        df.to_csv(os.path.join(dataset_dir,'labels.csv'), index=False)

    def _get_df_output(self, response_content: str) -> DataFrame:
        compressed_bytes = io.BytesIO(response_content)
        
        with tarfile.open(fileobj=compressed_bytes, mode="r:gz") as tar:
            output_tar_member = tar.getmember(self.output_path)
            output_file = tar.extractfile(output_tar_member)
            output_df = pd.read_json(io.BytesIO(output_file.read()), lines=True)       
        return output_df

    def search_dataset(self, dataset_name):
        url = self.base_url + self.datasets_url + '/search'
        params = {
            'dataset_name': dataset_name
        }
        response = self._get_call(url, params, f'Dataset {dataset_name} found in SambaStudio')
        parsed_reponse = json.loads(response.text)
        return parsed_reponse['data']['dataset_id']

    def delete_dataset(self, dataset_name):
        dataset_id = self.search_dataset(dataset_name)
        url = self.base_url + self.datasets_url + '/' + dataset_id
        response = self._delete_call(url)
        logging.info(response.text)
        
        
    def create_dataset(self, path):
        # create clip directory and source.json file
        
        dataset_name = f'{self.dataset_name}_{int(time.time())}'
            
        clip_directory = os.path.join(self.datasets_path, dataset_name)
        
        if not os.path.isdir(self.datasets_path):
            os.mkdir(self.datasets_path) 
            
        if not os.path.isdir(clip_directory):
            logging.info(f'Datasets path: {clip_directory} dont \'t found')
            
            source_file_data = {
                "source_path": clip_directory
            }
            
            with open(self.dataset_source_file, 'w') as json_file:
                json.dump(source_file_data, json_file)

        shutil.copytree(path, clip_directory)
        
        self._generate_csv(clip_directory)
        
        # create dataset
        command = f'echo yes | snapi dataset add \
            --dataset-name {dataset_name} \
            --job_type {self.job_type} \
            --apps {self.clip_app_id} \
            --source_type {self.dataset_source_type} \
            --source_file {self.dataset_source_file} \
            --description "{self.dataset_description}"'
        
        os.system(command)
        logging.info(f'Creating dataset: {dataset_name}')
        
        return dataset_name
         
    def check_dataset_creation_progress(self, dataset_name):
        url = self.base_url + self.datasets_url + '/' + dataset_name
        response = self._get_call(url)
        if response.json()["data"]["status"]=="Available": 
            return True
        else:
            return False
            
    def create_load_project(self):

        url = self.base_url + self.projects_url + '/' + self.project_name

        response = self._get_call(url, success_message=f'Project {self.project_name} found in SambaStudio')
        not_found_error_message = f"{self.project_name} not found"

        if not_found_error_message in response.text:
            
            logging.info(f'Project {self.project_name} wasn\'t found in SambaStudio')
            
            url = self.base_url + self.projects_url

            params = {
                'project_name': self.project_name,
                'description': self.project_description
            }

            response = self._post_call(url, params, success_message=f'Project {self.project_name} created!')

        parsed_reponse = json.loads(response.text)
        self.project_id = parsed_reponse['data']['project_id']
        return self.project_id
    
    def run_job(self, dataset_name):
        
        url = self.base_url + self.projects_url + self.jobs_url.format(project_id=self.project_id)
        
        params = {
            'task': self.job_task,
            'job_type': self.job_type,
            'job_name': f'{self.job_name}_{int(time.time())}',
            'project': self.project_id,
            'model_checkpoint': self.model_checkpoint,
            'description': self.job_description,
            'dataset': dataset_name,
        }

        response = self._post_call(url, params, success_message='Job running')
        parsed_reponse = json.loads(response.text)
        job_id = parsed_reponse['data']['job_id']
        
        return job_id
    
    def check_job_progress(self, job_id):
        """Check job progress of a given job.

        Args:
            job_id (str): The id of the job to check.
            
        Returns:
            bool: True when the job is finished.
        """

        url = self.base_url + self.projects_url + self.jobs_url.format(project_id=self.project_id) + '/' + job_id

        status = PENDING_RDU_JOB_STATUS
        while status != SUCCESS_JOB_STATUS:
            response = self._get_call(url, success_message='Still waiting for job to finish')
            parsed_reponse = json.loads(response.text)   
            status = parsed_reponse['data']['status']
            logging.info(f'Job status: {status}')
            if status == SUCCESS_JOB_STATUS:
                logging.info('Job finished!')
                break
            elif status == FAILED_JOB_STATUS:
                logging.info('Job failed!')
                return False
            time.sleep(10)
        
        return True  
    
    def delete_job(self, job_id):
        url = self.base_url +  self.projects_url + self.jobs_url.format(project_id=self.project_id) + '/' + job_id
        response = self._delete_call(url)
        logging.info(response.text)
        
    def retrieve_results(self, job_id):
        url = self.base_url + self.projects_url + self.jobs_url.format(project_id=self.project_id) + '/' + job_id + self.download_results_url
        response = self._get_call(url, success_message='Results downloaded!')
        df = self._get_df_output(response.content)
        return df

In [ ]:
clip = BatchClipProcessor(config)

In [ ]:
clip.create_load_project()

In [ ]:
dataset_name = clip.create_dataset(path=os.path.join(kit_dir,'data/images'))
while not clip.check_dataset_creation_progress(dataset_name):
    print("waiting for dataset available")
    time.sleep(1)

In [ ]:
clip.search_dataset(dataset_name)

In [ ]:
job_id = clip.run_job(dataset_name)

In [ ]:
result = clip.check_job_progress(job_id) 

In [ ]:
df=clip.retrieve_results(job_id)

In [ ]:
clip.delete_job(job_id)

In [ ]:
clip.delete_dataset(dataset_name)

In [ ]:
df.head()

## Use inference endpoint

#### Send image

In [ ]:
def embed_image(img_path= None, img=None):
    base_url = os.environ.get("BASE_URL")
    api_key = os.environ.get("API_KEY")
    project_id = os.environ.get("PROJECT_ID")
    endpoint_id = os.environ.get("ENDPOINT_ID") 
    url = f"{base_url}/api/predict/file/{project_id}/{endpoint_id}"
    if img_path:
        files = {'predict_file': open(img_path, 'rb')}
    elif img: 
        files = {'predict_file': img}
    else:
        raise Exception("please provide a image path or a bytes image file")
    headers = {'key': api_key}
    response = requests.post(url, files=files, headers=headers)
    return response.json()["data"][0]
    
jpg_file_path = os.path.join(kit_dir,"data/images/animals/animals_0.png")
print(embed_image(jpg_file_path))

#### Send Text

In [ ]:
def embed_text(text):
    base_url = os.environ.get("BASE_URL")
    api_key = os.environ.get("API_KEY")
    project_id = os.environ.get("PROJECT_ID")
    endpoint_id = os.environ.get("ENDPOINT_ID") 
    url = f"{base_url}/api/predict/nlp/{project_id}/{endpoint_id}"
    input_data = {"inputs": [text]}
    headers = {'key': api_key, 'Content-Type': 'application/json'}
    response = requests.post(url, json=input_data, headers=headers)
    return response.json()["data"][0]
    
text = "lion in the jungle"
print(embed_text(text))


In [ ]:

from chromadb.api.types import is_image, is_document, Images,  Documents, EmbeddingFunction, Embeddings, Protocol
from typing import cast, Union, TypeVar

## chromadb multimodal

In [ ]:
Embeddable = Union[Documents, Images]
D = TypeVar("D", bound=Embeddable, contravariant=True)

class ClipEmbbeding(EmbeddingFunction[D]):
    def __init__(self) -> None:
        pass
    def __call__(self, input: D) -> Embeddings:
        embeddings: Embeddings = []
        for item in input:     
            if is_document(item):
                output = embed_text(item)
            elif is_image(item):
                image = Image.fromarray(item)
                buffer = io.BytesIO()
                image.save(buffer, format='PNG')
                output = embed_image(img=buffer.getvalue())
            embeddings.append(output)
        return cast(Embeddings, embeddings)


In [ ]:
client = chromadb.PersistentClient(path=os.path.join(kit_dir,"data"))
clip_embedding=ClipEmbbeding()
try:
    client.delete_collection(name="image_collection")
except:
    pass
collection=client.get_or_create_collection(name="image_collection", embedding_function=clip_embedding, metadata={"hnsw:space": "l2"})
collection.get()


## Add individual images 

In [ ]:
def get_images(folder_path):
    images=[]
    paths=[]
    for root, _dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
                path=os.path.join(root, file)
                paths.append(path)
                image= np.array(Image.open(os.path.join(root, file)))
                images.append(image)
    return paths,images

In [ ]:
paths, images=get_images(os.path.join(kit_dir,"data/images"))
print(len(paths))

In [ ]:
collection.add(
    images=images,
    metadatas=[{"source": path} for path in paths],
    ids=paths,
    uris=paths
)

In [ ]:
collection.get()#include=["uris","documents"])

## Add batch preprocessed images

In [ ]:
embeddings = list(df["predictions"]) 
paths = list(df["input"].apply(lambda x: os.path.join(kit_dir,'data/images',x)))

In [ ]:
collection.add(
    embeddings=embeddings,
    metadatas=[{"source": path} for path in paths],
    ids=paths,
    uris=paths
)

In [ ]:
collection.get()#include=["uris","documents"])

## Search methods

In [ ]:
def search_image_by_text(query,n=3):
    result=collection.query(query_texts=[query],include=["uris", "distances"],n_results=n)
    return result['uris'][0], result["distances"][0]

In [ ]:
def search_image_by_image(path,n=5):
    image= np.array(Image.open(path))
    result=collection.query(query_images=[image],include=["uris", "distances"],n_results=n)
    return result['uris'][0], result["distances"][0]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_images(image_paths, distances):
    num_images = len(image_paths)
    fig, axes = plt.subplots(1, num_images, figsize=(10*num_images, 10))
    
    for i, path in enumerate(image_paths):
        img = mpimg.imread(path)
        axes[i].imshow(img)
        axes[i].axis('off')
        axes[i].set_title(f'Image {i+1}, d={distances[i]}')
    
    plt.show()


In [ ]:
uris, distances = search_image_by_text("iron")
show_images(uris, distances)

In [ ]:
uris, distances = search_image_by_image("../download.jpeg")
uris.insert(0, "../download.jpeg")
distances.insert(0, 0)
show_images(uris, distances)